In [1]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Model 
from tensorflow.keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers
#from tokenizers import  BertWordPieceTokenizer
import pandas as pd
import numpy as np


C:\Users\kusha\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from transformers import TFDistilBertModel

# transformer = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased') 


from transformers import DistilBertTokenizer

# Load the correct tokenizer for distilbert-base-uncased
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
print(f"Tokenizer vocabulary size: {tokenizer.vocab_size}")  # Should be 30522




Tokenizer vocabulary size: 30522


In [3]:
train_data=pd.read_csv('C:/Users/kusha/OneDrive/Desktop/Task/Deep_Learning_Everyday/BERT_NLP/jigsaw-toxic-comment-train.csv') 
test_data=pd.read_csv('C:/Users/kusha/OneDrive/Desktop/Task/Deep_Learning_Everyday/BERT_NLP/test.csv')
valid_data=pd.read_csv('C:/Users/kusha/OneDrive/Desktop/Task/Deep_Learning_Everyday/BERT_NLP/validation.csv')

In [4]:
train_data 

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [7]:
# def fast_encode(texts,tokenizer,chunk_size=256,maxlen=192):
#     tokenizer.enable_truncation(maxlen)
#     tokenizer.enable_padding(length=maxlen)
#     all_ids=[]

#     for i in range(0,len(texts),chunk_size):
#         text_chunk=texts[i:i+chunk_size].tolist()
#         encs=tokenizer.encode_batch(text_chunk)
#         all_ids.extend([enc.ids for enc in encs])
    
#     return np.array(all_ids)




In [5]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=192):
    all_ids = []  # List to store token IDs
    all_masks = []  # List to store attention masks

    original_rows = len(texts)
    num_full_chunks = len(texts) // chunk_size
    
    
    processed_rows = 0
    # Process texts in chunks
    for i in range(0, num_full_chunks*chunk_size, chunk_size):
        text_chunk = texts[i:i + chunk_size].tolist()  # Get a chunk of texts
        
        # Tokenize with padding and truncation explicitly set
        encs = tokenizer(text_chunk, truncation=True, padding='max_length', max_length=maxlen, return_tensors='np')

        chunk_size_actual = len(text_chunk)
        chunk_output_size = encs['input_ids'].shape[0]
        
        print(f"Chunk {i // chunk_size} - Input size: {chunk_size_actual}, Tokenized size: {chunk_output_size}")

        processed_rows += chunk_output_size  # Track processed rows


        # Extract `input_ids` and `attention_mask`
        all_ids.extend(encs['input_ids'])
        all_masks.extend(encs['attention_mask'])

    # Convert to NumPy arrays for easier handling
    return np.array(all_ids), np.array(all_masks)


#tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Example Usage
x_train,train_mask = fast_encode(train_data.comment_text.astype(str), tokenizer, maxlen=192)
x_valid,valid_mask = fast_encode(valid_data.comment_text.astype(str), tokenizer, maxlen=192)
x_test,test_mask = fast_encode(test_data.content.astype(str), tokenizer, maxlen=192)



Chunk 0 - Input size: 256, Tokenized size: 256
Chunk 1 - Input size: 256, Tokenized size: 256
Chunk 2 - Input size: 256, Tokenized size: 256
Chunk 3 - Input size: 256, Tokenized size: 256
Chunk 4 - Input size: 256, Tokenized size: 256
Chunk 5 - Input size: 256, Tokenized size: 256
Chunk 6 - Input size: 256, Tokenized size: 256
Chunk 7 - Input size: 256, Tokenized size: 256
Chunk 8 - Input size: 256, Tokenized size: 256
Chunk 9 - Input size: 256, Tokenized size: 256
Chunk 10 - Input size: 256, Tokenized size: 256
Chunk 11 - Input size: 256, Tokenized size: 256
Chunk 12 - Input size: 256, Tokenized size: 256
Chunk 13 - Input size: 256, Tokenized size: 256
Chunk 14 - Input size: 256, Tokenized size: 256
Chunk 15 - Input size: 256, Tokenized size: 256
Chunk 16 - Input size: 256, Tokenized size: 256
Chunk 17 - Input size: 256, Tokenized size: 256
Chunk 18 - Input size: 256, Tokenized size: 256
Chunk 19 - Input size: 256, Tokenized size: 256
Chunk 20 - Input size: 256, Tokenized size: 256
Ch

In [46]:
x_train.shape

(223488, 192)

In [48]:
print(f"Original train data size: {len(train_data.comment_text)}")
print(f"Encoded x_train size: {len(x_train)}")  # Should match


Original train data size: 223549
Encoded x_train size: 223488


In [6]:
AUTO=tf.data.experimental.AUTOTUNE

EPOCHS=3
BATCH_SIZE=256
MAX_LEN=192


In [7]:
# Train inputs
train_inputs = {
    'input_ids': x_train,
    'attention_mask': train_mask
}

# Validation inputs
valid_inputs = {
    'input_ids': x_valid,
    'attention_mask': valid_mask
}

# Test inputs
test_inputs = {
    'input_ids': x_test,
    'attention_mask': test_mask
}


In [8]:
y_train=train_data['toxic'].values

y_train_clipped=y_train[:x_train.shape[0]]


y_valid=valid_data['toxic'].values

y_valid_clipped=y_valid[:x_valid.shape[0]]




In [9]:
train_dataset=(tf.data.Dataset.from_tensor_slices((train_inputs,y_train_clipped)).repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO))

valid_dataset=(tf.data.Dataset.from_tensor_slices((valid_inputs,y_valid_clipped)).repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO))

In [10]:
test_dataset=(tf.data.Dataset.from_tensor_slices(x_test).batch(BATCH_SIZE))


In [161]:
# # def build_model(transformer,max_len=192):
# #     # input_word_ids=Input(shape=(max_len,),dtype=tf.int32,name='input_word_ids')
# #     # sequence_output=transformer(input_word_ids)[0]
# #     input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_word_ids')
# #     attention_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')
# #     sequence_output = transformer(input_ids=input_word_ids, attention_mask=attention_mask)[0]
# #     cls_token=sequence_output[:,0,:]
# #     out=Dense(1,activation='sigmoid')(cls_token)

# #     model=Model(inputs=[input_word_ids,attention_mask],outputs=out)
# #     model.compile(Adam(lr=1e-5),loss='binary_crossentropy',metrics=['accuracy'])

# #     return model








# def build_model(transformer_func, max_len=192):
#     # Input layers
#     input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_word_ids')
#     attention_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')
#     # Pass inputs to the transformer
#     # sequence_output = transformer(input_ids=input_word_ids, attention_mask=attention_mask)[0]
#     print(f"Input Word IDs: {input_word_ids.shape}")
#     print(f"Attention Mask: {attention_mask.shape}")
#     transformer_output = transformer(input_ids=input_word_ids, attention_mask=attention_mask)
#     print(f"Transformer Output: {transformer_output['last_hidden_state'].shape}")
 
    
#     # transformer_output = transformer(input_ids=input_word_ids, attention_mask=attention_mask)
#     # print(transformer_output)
#     sequence_output = transformer_output.last_hidden_state
#     print(sequence_output)
#     # Extract the CLS token
#     cls_token = sequence_output[:, 0, :]

#     # Output layer for binary classification
#     out = Dense(1, activation='sigmoid')(cls_token)

#     # Define the model
#     model = Model(inputs=[input_word_ids, attention_mask], outputs=out)
    
#     # Compile the model
#     model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

#     return model

    
    
    



In [232]:
# from transformers import TFDistilBertModel
# import tensorflow as tf

# # Initialize transformer model
# transformer = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
# print(transformer) 

# #Define input layers
# # max_len = 192  # Example length, adjust as needed
# # input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name='input_word_ids')
# # attention_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')
# # print(input_word_ids)
# # # Get transformer output (correct usage of call method)
# #transformer_output = transformer(input_ids=input_word_ids, attention_mask=attention_mask)
# transformer_output = transformer(x_train, attention_masks_train)

# # # Extract last hidden state
# # sequence_output = transformer_output['last_hidden_state']

# # # Get [CLS] token for classification
# # cls_token = sequence_output[:, 0, :]  # Shape: (batch_size, hidden_size)

# # # Define output layer (for binary classification)
# # out = tf.keras.layers.Dense(1, activation='sigmoid')(cls_token)

# # # Build and compile model
# # model = tf.keras.Model(inputs=[input_word_ids, attention_mask], outputs=out)
# # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # model.summary()


In [233]:
# transformer_layer=(transformers.TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased'))
# model=build_model(transformer_layer,max_len=192)
# model.summary 


# model = build_model(transformer,max_len=max_len)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.summary()





In [11]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [12]:
import tensorflow as tf

# Create a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': x_train,
    'attention_mask': train_mask
}, y_train_clipped)).batch(BATCH_SIZE)

In [13]:
from tensorflow.keras.optimizers import Adam

# Define optimizer and loss function
optimizer = Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset, epochs=3)

Epoch 1/3




In [ ]:
n_steps=x_train.shape[0]//BATCH_SIZE

train_history=model.fit(train_dataset,steps_per_epoch=n_steps,validation_data=valid_dataset,epochs=EPOCHS)


In [ ]:
n_steps=x_valid.shape[0]//BATCH_SIZE
train_history_2=model.fit(valid_dataset.repeat(),steps_per_epoch=n_steps,epochs=EPOCHS*2)

predictions=model.predict(test_dataset,verbose=1)
